In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Load the LFW dataset using scikit-learn's fetch_lfw_people
from sklearn.datasets import fetch_lfw_people

# Load the LFW dataset
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
n_samples, h, w = lfw_people.images.shape
X = lfw_people.data
n_features = X.shape[1]
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

# Define a CNN-based classifier
class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * ((h - 4) // 2) * ((w - 4) // 2), 128)  # Adjust input size based on your data dimensions
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.view(-1, 1, h, w)  # Reshape input for convolutional layers
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)  # Max pooling
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the CNN classifier
cnn_classifier = CNNClassifier(num_classes=len(target_names))

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_classifier.parameters(), lr=0.001)

# Training loop
batch_size = 64
num_epochs = 11

for epoch in range(num_epochs):
    cnn_classifier.train()
    total_loss = 0.0
    for i in range(0, len(X_train), batch_size):
        inputs = X_train[i:i+batch_size].unsqueeze(1)
        labels = y_train[i:i+batch_size]

        optimizer.zero_grad()
        outputs = cnn_classifier(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / (len(X_train) / batch_size)}")

# Testing loop
cnn_classifier.eval()
with torch.no_grad():
    test_inputs = X_test.unsqueeze(1)
    test_outputs = cnn_classifier(test_inputs)
    _, predicted = torch.max(test_outputs, 1)
    print(classification_report(y_test, predicted.numpy(), target_names=target_names))
   



Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


TypeError: CNNClassifier.__init__() got an unexpected keyword argument 'num_classes'